In [6]:
!dir

ASRManager.py  ASRManagerOld.py  __pycache__  api_service.py  whisper-converted


In [1]:
from nfASRManager import ASRManager
import numpy as np
import io
import librosa

import pyloudnorm as pyln

In [2]:
myManager = ASRManager()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
filenames = ["tester.wav"]*4 

frequency = 16000
wf = []
wfn =[]
for f in filenames:
    waveform, sr = librosa.load(f, sr = frequency)
    
    pna = pyln.normalize.peak(waveform, 1.0)
    meter = pyln.Meter(frequency) # create BS.1770 meter
    loudness = meter.integrated_loudness(pna)
    pna = pyln.normalize.loudness(pna, loudness, 0.0)
    wf.append(pna)
    wfn.append(waveform)

/opt/conda/lib/python3.10/site-packages/pyloudnorm/normalize.py:31: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/opt/conda/lib/python3.10/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/opt/conda/lib/python3.10/site-packages/pyloudnorm/normalize.py:31: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/opt/conda/lib/python3.10/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/opt/conda/lib/python3.10/site-packages/pyloudnorm/normalize.py:31: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/opt/conda/lib/python3.10/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possib

In [3]:
wf

[array([ 0.0736966 , -0.07317418,  0.19912611, ..., -0.03649745,
        -0.10859019, -0.02184238], dtype=float32),
 array([ 0.0736966 , -0.07317418,  0.19912611, ..., -0.03649745,
        -0.10859019, -0.02184238], dtype=float32),
 array([ 0.0736966 , -0.07317418,  0.19912611, ..., -0.03649745,
        -0.10859019, -0.02184238], dtype=float32),
 array([ 0.0736966 , -0.07317418,  0.19912611, ..., -0.03649745,
        -0.10859019, -0.02184238], dtype=float32)]

In [5]:
wfn

[array([ 0.00156825, -0.00155713,  0.00423736, ..., -0.00077666,
        -0.00231077, -0.0004648 ], dtype=float32),
 array([ 0.0000000e+00,  0.0000000e+00,  3.0517578e-05, ...,
        -9.1552734e-04,  4.8828125e-04,  1.8005371e-03], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([ 0.0000000e+00,  0.0000000e+00,  3.0517578e-05, ...,
        -9.1552734e-04,  4.8828125e-04,  1.8005371e-03], dtype=float32)]

In [4]:
myManager.batch_transcribe(wf)

[' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.',
 ' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.',
 ' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.',
 ' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.']

In [5]:
myManager.batch_transcribe_vad(wf, vad_ms=500)

ValueError: <|startoftranscript|> token was not found in the prompt

In [12]:
with open(filename, "rb") as file:
        audio_bytes = file.read()

In [16]:
with open("bytes.txt", "w") as f:
    f.write(str(audio_bytes))

In [25]:
w, s = librosa.load(io.BytesIO(audio_bytes), sr = 16000)


In [8]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

# Single 

In [4]:
import os
os.environ['HF_HOME'] = 'huggingface'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = 'True'
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
weights = "model"

# print(np.array(waveform, dtype= float))
pipe = pipeline(task="automatic-speech-recognition", model=weights, device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
answers = [x['text'] for x in pipe(wf)]

In [10]:
answers

[' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.',
 ' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.',
 ' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.',
 ' Turrets, prepare to deploy electromagnetic pulse, heading zero seven nine. Target is yellow fighter jet. Engage.']

In [7]:
from transformers import WhisperProcessor
import numpy as np
import os
os.environ['HF_HOME'] = 'huggingface'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = 'True'
# import torch
# import datasets
# from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset


In [9]:
weights = "model"

processor = WhisperProcessor.from_pretrained('openai/whisper-small.en',resume_download = None)
processor.tokenizer.save_pretrained(weights)
processor.feature_extractor.save_pretrained(weights)
transcriber = pipeline("automatic-speech-recognition", model=weights, device=0)

SafetensorError: Error while deserializing header: MetadataIncompleteBuffer

In [ ]:
def batch_transcribe(self, batch) -> str:
        # batch is a list of audio waveforms
        # This is extremely rudimentary. Currently looking at possible batch inference using faster-whisper
        batchResponse = []
        for wf in batch:
            output = ""
            # segments, _ = self.model.transcribe(wf, beam_size = 5, vad_filter = True)
            # segments, _ = self.model.transcribe(wf, beam_size = 5)
            for s in segments:
                output += s.text
            batchResponse.append(output)
            
        return batchResponse